# Amazing RL Agent

In [178]:
import gymnasium as gym
import torch
import numpy as np
import importlib

# Render screen, set to false if training
RENDER_AGENT = True


In [179]:
# Test pytorch is working

x = torch.rand(size=(1000,))
x = x + 1

# If you have a nvidia gpu with torch installed correctly
# you can move the tensors to the gpu for some extra
# speed, not necessary although.
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")
# When creating a new tensor make sure you move it to a
# device using .to(DEVICE), such as:
x = x.to(DEVICE)
# The tensor in x has been moved to the device


Using cpu


In [180]:
# Import agent
import agents

# Reload module as jupyter notebooks wont update to any changes youve made
importlib.reload(
    agents
)
#agent = agents.Agent()

# Action space is currently 12 because you cant actually apply DQN to continuous action problems
# 24 + 4 as the controller state is included
agent = agents.DQN(24 + 4, 12, mini_batch_len=2**10)
control = agents.Controller(4, 0.1)


In [181]:
env = gym.make("BipedalWalker-v3", hardcore=False, render_mode="human")
observation, info = env.reset(seed=27)
steps_per_episode = 200
for episode in range(100000):
    rewards = 0
    step = 0
    while(step < steps_per_episode):
        action = agent.choose_action(
            env.action_space, np.concatenate((observation, control.vals))
        )  # this is where you would insert your policy
        control.update(action)
        observation, reward, terminated, truncated, info = env.step(control.vals)
        rewards += reward

        agent.update(np.concatenate((observation, control.vals)), reward, terminated, truncated)
        if terminated or truncated:
            observation, info = env.reset()
            control.reset()
        step += 1
        env.render()
    print(episode, rewards)
    observation, info = env.reset()
    control.reset()
env.close()


KeyboardInterrupt: 